In [1]:
import os
import pandas as pd
from datetime import datetime, timedelta

In [2]:
# 格納フォルダ
fileDir = r'C:\Users\u647088\Desktop'

# 投薬データファイル
inFile = 'yakuzaiData.csv' 


In [7]:
def extractLine(df):
    def __extractLine(tDf, kai, syu):
        # 薬剤ごとの期間の算出
        sDf = tDf[
            (((kai <= tDf['kaibi']) & (tDf['kaibi'] <= syu)))].groupby(
                'yakuzai', as_index=False).agg(
                    {'kaibi': 'min', 'syubi': 'max'}).copy()
        # 開始日による暫定治療ラインの分割
        periodSet = set(sDf['kaibi'].tolist())
        tSet = set()
        for tDate in periodSet:
            # 開始日が暫定治療ライン内に存在したら分割
            if kai < tDate <= syu and kai != tDate:
                tSet = tSet | __extractLine(
                    tDf, kai, tDate + timedelta(days=-1))
                tSet = tSet | __extractLine(tDf, tDate, syu)
        if len(tSet) == 0:
            tSet.add((kai, syu))
        return tSet

    # 全体の期間の算出
    tDf = df.copy()
    tDf['kaibi'] = pd.to_datetime(tDf['kaibi'], format='%Y%m%d')
    tDf['syubi'] = pd.to_datetime(tDf['syubi'], format='%Y%m%d')

    kaibi = tDf['kaibi'].min()
    syubi = tDf['syubi'].max()

    datelist = __extractLine(tDf, kaibi, syubi)

    # 日付の文字列化、開始日の昇順でソート
    datelist = [(kai.strftime('%Y%m%d'),
                 syu.strftime('%Y%m%d')) for kai, syu in datelist]
    datelist = sorted(datelist, key=lambda x: x[0])

    # 治療ラインごとの集計
    rDf = pd.DataFrame()
    for i, l in enumerate(datelist):
        # 治療ラインの期間内に投薬開始日となっている薬剤で集計
        tDf = df[(((l[0] <= df['kaibi']) & (df['kaibi'] <= l[1])))].copy()
        tDf = tDf.groupby('yakuzai', as_index=False).agg(
            {'kaibi': 'min', 'syubi': 'max'})
        tDf['治療ライン'] = i + 1
        tDf['同一治療ライン上の薬剤数'] = len(tDf)
        # 薬剤の投薬終了日が治療ラインの投薬終了日より後になっていた場合、
        # 当治療ラインの終了日にする
        tDf['syubi'] = tDf['syubi'].apply(
            lambda x: l[1] if l[1] < x else x)

        # 最初の治療ライン以外の場合
        if i != 0:
            # 前の治療ラインに同一薬剤が存在したら「確認対象」とする
            for yakuzai in tDf['yakuzai']:
                cDf = tDf[tDf['yakuzai'] == yakuzai].copy()
                if len(rDf[rDf['yakuzai'] == yakuzai]) != 0:
                    cDf['確認対象フラグ'] = '確認対象'
                else:
                    cDf['確認対象フラグ'] = '確認対象外'
                rDf = pd.concat([rDf, cDf])
        else:
            tDf['確認対象フラグ'] = '確認対象外'
            rDf = pd.concat([rDf, tDf])

    return rDf

In [8]:
"""
**メイン処理**

# テストデータの読込
# 対象データ抽出（CSV読み込み）
# 患者ごとの治療ライン導出処理実行
# 処理結果ファイルの出力
"""

# テストデータの読込
dataFile = os.path.join(fileDir, inFile)
df = pd.read_csv(dataFile, dtype=str, encoding='cp932')
df = df.fillna('')
# 患者ごとの処理実行(施設名が'__'を含まないことと'_'で終わらないことが前提)
df['kanja'] = df.apply(
    lambda row: str(row['facility']) + '__' + str(row['kanja_id']),
    axis=1)
rDf = pd.DataFrame()
for kanja in df[~df['kanja'].duplicated()]['kanja']:
    tDf = extractLine(df[df['kanja'] == kanja].copy().drop(
        ['kanja', 'kanja_id', 'facility'], axis=1))
    dKanja = kanja.split('__')
    tDf['facility'] = dKanja[0]
    tDf['kanja_id'] = dKanja[1]
    rDf = pd.concat([rDf, tDf])

# 結果出力
rDf.sort_values(
    ['facility', 'kanja_id', '治療ライン', 'kaibi', 'yakuzai'],
    inplace=True)
# 処理結果ファイルの出力
file = os.path.join(fileDir, 'yakuzaiLine_' + "{0:%Y%m%d%H%M}".format(datetime.now()) + '.tsv')
rDf.to_csv(file, encoding='cp932', sep='\t', index=False, header=True,
           columns=['facility', 'kanja_id', 'yakuzai', 'kaibi',
                    'syubi', '治療ライン', '確認対象フラグ',
                    '同一治療ライン上の薬剤数'])


       yakuzai      kaibi      syubi
0  乾燥ＢＣＧ膀胱内用_1 2021-01-20 2021-03-07
1  乾燥ＢＣＧ膀胱内用_2 2021-03-08 2021-03-28
2  乾燥ＢＣＧ膀胱内用_3 2021-03-29 2021-05-30
